# Introduction
This project goal is finding a answers some question like, how is the change footplayer market value and how the feature effect that. 
Our dataset have 74 features and 18207 number of examples. Firstly, I will try to analyze dataset and make a visualazition before the machine learning model. After this step we will understand much better our dataset and maybe some of the feauteres are not necessary for us also some of them need to be convert string to integer. 


# Step 0 - Load Data
Reading data and checking initial feature. Also I will import some library that are necessary.

In [1]:
#some useful library and necessary things
import pandas as pd
import numpy as np 


from scipy.stats import pearsonr

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display



In [3]:
#Read data from .csv file

f_players = pd.read_csv("train.csv")

display(f_players.head())
print(f"Total number of rows: {f_players.shape[0]}\nTotal number of columns: {f_players.shape[1]}")



,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Total number of rows: 18207
Total number of columns: 89


As you can see above there are some useless features for us. We can drop these are on dataset and in this way, we can study more efficiently.These are unnecessary for us. ID,Name,Photo,Flag,Club logo,Real Face,Jersey Number,Joined, Realese Clause and Loaned from

In [ ]:
display(fp.head())
print(f"Total number of rows: {fp.shape[0]}\nTotal number of columns: {fp.shape[1]}")


In [ ]:

# fp is our new data set. 
drop_columns = [ 'Unnamed: 0', 'ID','Nationality','Name', 'Photo','Preferred Foot', 'Flag','Club Logo', 'Body Type', 'Real Face', 'Jersey Number','Joined', 'Loaned From','Release Clause','Contract Valid Until']
fp = f_players.copy()
fp.drop(drop_columns, axis =1 , inplace = True)
#fp.drop(['Unnamed: 0,'ID', 'Name', 'Photo','Flag','Club Logo','Body Type', 'Real Face','Jersey Number','Joined', 'Loaned From','Release Clause','Contract Valid Until'], axis=1, inplace=True)
#fp.drop(fp.columns[[0]], axis=1,inplace = True)

In [ ]:
" Before the analysis value we need to change type of Value to float."
" Example: Kevin de Bruyne Value = €102M	 €102*1000000 and wage= €355K	 €355000"

def str2float(amount):
    """
    This function help to us for convert string to float.
       
    """
   
    if amount[-1] == 'M':
        return float (amount[1:-1])*1000000
    elif amount[-1] == 'K':
        return float (amount[1:-1])*1000
    else:
        return float(amount[1:])
    
fp['Value_Float'] = fp['Value'].apply(lambda x: str2float(x))
fp['Wage_Float'] = fp ['Wage'].apply(lambda x: str2float(x))

#Now we do not need string value and wage. We can drop them

fp.drop(['Value','Wage'] ,axis =1 , inplace = True)

In [ ]:
display(fp.head())
print(f"Total number of rows: {fp.shape[0]}\nTotal number of columns: {fp.shape[1]}")

In [ ]:
display(fp.isnull().sum())

plt.show()

In [ ]:
## Club has a 241 missing and GKKicking 48, and we are dropping these missing datapoints

fp = fp.loc[fp['Club'].notnull()].copy()
fp = fp.loc[fp['GKKicking'].notnull()].copy()


In [ ]:
display(fp.isnull().sum())

# Step 1
Value (Target Datapoints).

Mainly we are focusing on players market values. On this part, we will looking this area.

In [ ]:
def distribution_and_stats_plot(variable,Name):
    """For better understanding for players value distribution.I will draw a plot charts"""
    
    fig, ax = plt.subplots(nrows=2,figsize =(15,12),gridspec_kw={'height_ratios': [2,1]})
    sns.distplot(variable, ax = ax[0])
    ax[0].set(title = f"Distribution of {Name}",xlabel='',ylabel='')
    ax[0].set_yticks([])
    
    ax[1]=sns.boxplot(x=variable.columns[0],y=None,data=variable)
    ax[1].set_title(f"Boxplot of {Name}")
    plt.show()
    #sns.boxplot(x="distance", y="method", data=planets,  whis=[0, 100], width=.6, palette="vlag")
   


    
    display(variable.describe())
    
distribution_and_stats_plot(fp[["Value_Float"]]/1000000,'Player Values (\N{euro sign}M)')



### Step 1.1

#### Player  Value
As you can see the above, our players generally have low market value and for this reason, we can not see general 
the distribution of that. High prices players are almost not seeing on the chart. For that, we can use a logarithmic distribution chart, it shows us more clearly.

In [ ]:
# In my data set there are 11 values equal 0 that is why, I added plus 1 for every values for converting log 
# After that I reaileze that 11 players old person and low overall rating. They do not any good affect on model.
# For this reason I deleted them and now my graphs is better than befor more clear and understandable.
#
#Convert to log

fp["Value_Float_log"] = np.log(fp["Value_Float"]+1)

#delete value_log=0 
fp =fp[fp["Value_Float_log"] !=0]

distribution_and_stats_plot(fp[["Value_Float_log"]],'Logaritmic Value')


### Step 1.2
#### Age
The step where the players are shown to the age distribution!!

In [ ]:
distribution_and_stats_plot(fp[["Age"]],'Player Ages')

In [ ]:
from scipy.stats import pearsonr

plt.scatter(x=fp["Age"],y=fp["Value_Float_log"],alpha=0.2)
plt.suptitle("Age vs Value",fontsize=15,y=0.95)
plt.xlabel("Age")
plt.ylabel("Value")
plt.show()

print(f"Pearson correlation = {pearsonr(fp['Value_Float'],fp['Age'])[0]}")

### Step 1.3

#### Player Position


In [ ]:
fp['Position'].value_counts(normalize= True).plot(kind='bar')
plt.title("Distribution of Football Players according to their positions.")
plt.ylabel("Ratio")
plt.show()

print(f"Number of positions= {len(fp['Position'].unique())}")

As you can see the above graphic, the most common positions are ST and GK. 

We have 27 unique positions. It's too much for analysis that's why we can classify these as, Attacker, midfielder, defender and goalkeeper. For that I will create new function.

In [ ]:
def classify_position(position):
    
    Position_dict= { 'Attack' : ['ST','LS','RS','CF','RF','LF'],
                    'Midfield': ['CM','RM','LM','CAM','CDM','LCM','RCM','LW','RW','RDM','LDM','LAM','RAM'],
                     'Defence': ['CB','LB','RB','RCB','LCB','RWB','LWB'],
                   'GoalKeeper':['GK']}
    
    for group , position_list in Position_dict.items():
        
        if position in position_list:
            return group
    
    return np.nan

In [ ]:
fp['Position_classify'] = fp['Position'].apply(classify_position)

## We classify our dataset, and we do not need fp['Position']
fp.drop(['Position'] ,axis =1 , inplace = True)

In [ ]:
fp['Position_classify'].value_counts(normalize= True).plot(kind='bar')
plt.title("Distribution of Football Players according to their positions.")
plt.ylabel("Ratio")
plt.show()

##This chart shows a clearer distribution of players' positions.

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x="Value_Float_log",y="Position_classify",data=fp)
plt.title("BoxPlot: logarithmic value by positions")
plt.show()

As you can see above, the distribution's Daily Market Value is very close to each. Attacker and Midfield players have average highs value and the goalkeeper average low, which is expected.

### Step 1.4

#### Player Potential

In this section, we will examine the distribution of the predicted potential of football players.

In [ ]:
distribution_and_stats_plot(fp[["Potential"]],'Player Potential')

The chart is more or less normally distributed and with a mean 71.32.

Also, we can examine in detail the player potential according to their positions.

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x="Potential",y="Position_classify",data=fp)
plt.title("BoxPlot of player potential by position")
plt.show()

The potential of player does not seem much different for each position. Next step, we will look relationship between player potential and player values. 

In [ ]:
sns.jointplot(x='Potential',y="Value_Float_log",data=fp,kind='reg')
plt.suptitle("Player Potential vs Value Log", fontsize=15,y=1.05)
plt.show()

The graphic turned out as expected. As you can see above, if a player has high potential it also has high market values. They are correlated.

This situation also expected between overall rating and potential. These are correlated. I wiil show them on next step.

In [ ]:
plt.scatter(x=fp["Potential"],y=fp["Overall"],alpha=0.2)
plt.suptitle("Potential vs Overal Rating",fontsize=15,y=0.95)
plt.xlabel("Potential Rating")
plt.ylabel("Overall Rating")
plt.show()

print(f"Pearson correlation = {pearsonr(fp['Potential'],fp['Overall'])[0]}")

You can see the above graphs, potential rating is higher than overall rating, that makes sense. But correlation is not enough for the eliminate one feature, for this reason I will keep both of them.

Next step, I will look difference between potential and overall rating. It can be a nice feature to show the effect of 
potential rating increase on market value.


# BUNU BELKİ SİLEBİLİRSİN SADECE YORUM YAPMAK İÇİN KULLANILIYOR 

In [ ]:
# Potential Growth
fp['PotentialGrowth'] = fp['Potential']-fp['Overall']

In [ ]:
fp['PotentialGrowth'].value_counts(normalize= True).plot(kind='bar')
plt.title("Distribution of Football Players Potential Growth.")
plt.xlabel("Potential Growth")
plt.ylabel("Ratio")
plt.show()

yorum yaz blalallalalalalallala




In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(x="PotentialGrowth",y="Value_Float_log",data=fp)
plt.suptitle("Potential Growth vs Value Log",fontsize=15,y=0.95)
plt.xlabel("Potential Growth ")
plt.ylabel("Value Log")
plt.show()

print(f"Pearson correlation = {pearsonr(fp['Value_Float'],fp['PotentialGrowth'])[0]}")

In [ ]:
plt.scatter(x=fp["PotentialGrowth"],y=fp["Overall"],alpha=0.2)
plt.suptitle("PotentialGrowth vs Overal Rating",fontsize=15,y=0.95)
plt.xlabel("PotentialGrowth")
plt.ylabel("Overall Rating")
plt.show()

print(f"Pearson correlation = {pearsonr(fp['PotentialGrowth'],fp['Overall'])[0]}")

In [ ]:
#### yorumm 

In [ ]:
#del fp['PotentialGrowth']

### Step 1.5

#### Player Overall Rating

In [ ]:
distribution_and_stats_plot(fp[["Overall"]],'Overall Rating')

Overal rating is normally distributed with a average of 66, minimum 46 and maximum 94.

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x="Overall",y="Position_classify",data=fp)
plt.title("BoxPlot of Overall Player Rating by position")
plt.show()

Except for Goalkeepers, other position more or less has the same overall rating. This interesting because as we see before on the logarithmic value by positions chart attackers and midfields market value higher than the others. 

In [ ]:
sns.jointplot(x='Overall',y="Value_Float_log",data=fp,kind='reg')
plt.suptitle("Overal Player Rating vs Value Log", fontsize=15,y=1.05)
plt.show()

print(f"Pearson correlation = {pearsonr(fp['Value_Float_log'],fp['Overall'])[0]}")
print(f"R2= {pearsonr(fp['Value_Float_log'],fp['Overall'])[0]**2}")

In [ ]:
### Yorummmm yapılabilir.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<b>Feature Correlations to the target variable, log(Value_Float)<b>"))
fp.corr()["Value_Float_log"].sort_values(ascending=False).iloc[1:].plot(kind='barh',figsize=(10,15))
plt.xlabel("Correlation with target variable")
plt.show()

# Step 2 
 
## Modeling 


#### 2.1 Data Pre.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

def loaddataset():
    """Datayı yükleeee"""
    
    #read and load initial data 
    f_players = pd.read_csv("train.csv")
    
    # fp is our new data set. 
    drop_columns = [ 'Unnamed: 0', 'ID','Name', 'Photo', 'Flag','Club Logo', 'Body Type', 'Real Face', 'Jersey Number','Joined', 'Loaned From','Release Clause','Contract Valid Until']
    fp = f_players.copy()
    fp.drop(drop_columns, axis =1 , inplace = True)
    
    #Change the type of wage and value
    fp['Value_Float'] = fp['Value'].apply(lambda x: str2float(x))
    fp['Wage_Float'] = fp ['Wage'].apply(lambda x: str2float(x))
    
    #Delete the prev value and wage
    fp.drop(['Value','Wage'] ,axis =1 , inplace = True)
    
    ## Dropping these missing datapoints
    fp = fp.loc[fp['Club'].notnull()].copy()
    fp = fp.loc[fp['GKKicking'].notnull()].copy()
    
    #Convert to log
    fp["Value_Float_log"] = np.log(fp["Value_Float"]+1)
    fp =fp[fp["Value_Float_log"] !=0]
    fp.drop(["Value_Float"] ,axis =1 , inplace = True)
    
    # Potential Growth
    fp['PotentialGrowth'] = fp['Potential']-fp['Overall']
    
    
    
    # Shuffled the dataframe list
    fp_sh = shuffle(fp, random_state = 0)
    
    #train and test data seperated
    return train_test_split( fp_sh.iloc[:,:-1],
                             fp_sh.iloc[:,-1],
                             test_size=0.33,
                             random_state=0)

In [ ]:
##training and final test sets
X_train, X_test, y_train, y_test = loaddataset()

In [ ]:
X_test

## Step 2.2 Baseline


In [ ]:
### yorummmmm

In [ ]:
#dictionary for the store results
results = {}

### r-squared

def calculate_r2(trainingdata,r2):
    
    x = len(trainingdata)
    y = len(trainingdata.columns)
    
    return 1-((1-r2)*(x-1)/(x-y-1))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

x = X_train ["Potential"].values.reshape(-1,1)
y = y_train.values.reshape(-1,1)

baseline_m = LinearRegression()
baseline_m.fit(x,y)

baseline_p = baseline_m.predict(X_train["Potential"].values.reshape(-1,1))
baseline_rmse = np.sqrt(mean_squared_error(y_train,baseline_p))
baseline_r2 = r2_score(y_train,baseline_p)
baseline_adj_r2 = calculate_r2(X_train[["Potential"]],baseline_r2)

###
results['Baseline Model- Linear Regression'] = {'RMSE':baseline_rmse, 'ADJ.R2':baseline_r2}

pd.DataFrame.from_dict(results,orient= 'index')

##### 222

## Step 2.3 Multiple

In [ ]:
## 
X_train_mult = pd.get_dummies(X_train,drop_first=True)
X_test_mult = pd.get_dummies(X_test,drop_first=True)

X_train_mult.head()

In [ ]:
import statsmodels.api as sm



In [ ]:
def gradientDescent(X,Y,theta,alpha,n_iteration):
    
    """RADIENTDESCENT Performs gradient descent to learn theta"""
    import numpy as np
    m = len(Y)
    J_hist = np.zeros((n_iteration,1))
    
    for iteration in range(n_iteration):
        h = np.dot(X,theta)
        
        for i in range(X.shape[1]):
            col = X.columns[i]
            theta[i]=theta[i]-alpha/m*sum(np.dot(X[col],(np.subtract(Y,h))))
            
        
        J_hist